In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.optimize import minimize
%matplotlib inline
import GPy as GPy

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
save_name_exo = input("insert the name of your exo: ")
print(save_name_exo)

insert the name of your exo: XSPINE
XSPINE


### Load data

In [3]:
#dataset without exo

Data_Past_Bystronic = 'subject-1.csv'
df_no_exo = pd.read_csv(Data_Past_Bystronic)
df_no_exo

,acc,accuracy,average_response_time,avg_rt,background,canvas_backend,clock_backend,color_backend,compensation,coordinates,...,time_prepare_SingleTask_1,time_sound_start_recording,time_sound_stop_recording,time_trial_sequence,title,total_correct,total_response_time,total_responses,uniform_coordinates,width
0,100.000000,100.000000,6251.000000,6251.000000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,25262,33097,25262,Dual Task,1,6251,1,yes,1024
1,50.000000,50.000000,3533.000000,3533.000000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,33365,35831,33365,Dual Task,1,7066,2,yes,1024
2,33.333333,33.333333,2536.666667,2536.666667,black,xpyriment,legacy,legacy,0,uniform,...,NaN,36084,38270,36084,Dual Task,1,7610,3,yes,1024
3,50.000000,50.000000,2108.750000,2108.750000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,38531,40681,38531,Dual Task,2,8435,4,yes,1024
4,60.000000,60.000000,1781.400000,1781.400000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,40937,42739,40937,Dual Task,3,8907,5,yes,1024
5,66.666667,66.666667,1598.333333,1598.333333,black,xpyriment,legacy,legacy,0,uniform,...,NaN,42987,45280,42987,Dual Task,4,9590,6,yes,1024
6,57.142857,57.142857,1441.571429,1441.571429,black,xpyriment,legacy,legacy,0,uniform,...,NaN,45522,47509,45522,Dual Task,4,10091,7,yes,1024
7,62.500000,62.500000,1336.500000,1336.500000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,47769,49926,47768,Dual Task,5,10692,8,yes,1024
8,66.666667,66.666667,1220.777778,1220.777778,black,xpyriment,legacy,legacy,0,uniform,...,NaN,47769,49926,47768,Dual Task,6,10987,9,yes,1024
9,60.000000,60.000000,1176.500000,1176.500000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,47769,49926,47768,Dual Task,6,11765,10,yes,1024


### Extraction of columns

In [4]:
# allocate data of a column
# X=df.iloc[:,0:1].values

# print header of dataframe
# list(df.columns.values)


In [4]:
#total wrong asnwer

total_resp_df_no_exo = df_no_exo.filter(regex='total_responses')
total_resp_correct_df_no_exo = df_no_exo.filter(regex='total_correct')

dim_df_no_exo = total_resp_df_no_exo.size

total_err_no_exo = total_resp_df_no_exo.iloc[dim_df_no_exo-1,0:1].values - total_resp_correct_df_no_exo.iloc[dim_df_no_exo-1,0:1].values

print(total_resp_df_no_exo.iloc[dim_df_no_exo-1,0:1].values)
print(total_resp_correct_df_no_exo.iloc[dim_df_no_exo-1,0:1].values)
print(total_err_no_exo)

total_resp_no_exo = df_no_exo.filter(regex='total_responses')

#total time: sum of times from question to answer

total_time_df_no_exo = df_no_exo.filter(regex='total_response_time')
total_time_no_exo = total_time_df_no_exo.iloc[dim_df_no_exo-1,0:1].values/1000 # [s]

print(total_time_no_exo)

#execution time: total time ascending/descending
exe_time_ad_no_exo = input("insert the total time for ascending/descendig without exo [s]: ")
float(exe_time_ad_no_exo)

print(exe_time_ad_no_exo)

[128]
[112]
[16]
[68.747]
insert the total time for ascending/descendig without exo [s]: 540.12
540.12


In [5]:
#dataset with exo

Data_Past_Bystronic = 'subject-1.csv'
df_exo = pd.read_csv(Data_Past_Bystronic)
df_exo

,acc,accuracy,average_response_time,avg_rt,background,canvas_backend,clock_backend,color_backend,compensation,coordinates,...,time_prepare_SingleTask_1,time_sound_start_recording,time_sound_stop_recording,time_trial_sequence,title,total_correct,total_response_time,total_responses,uniform_coordinates,width
0,100.000000,100.000000,6251.000000,6251.000000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,25262,33097,25262,Dual Task,1,6251,1,yes,1024
1,50.000000,50.000000,3533.000000,3533.000000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,33365,35831,33365,Dual Task,1,7066,2,yes,1024
2,33.333333,33.333333,2536.666667,2536.666667,black,xpyriment,legacy,legacy,0,uniform,...,NaN,36084,38270,36084,Dual Task,1,7610,3,yes,1024
3,50.000000,50.000000,2108.750000,2108.750000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,38531,40681,38531,Dual Task,2,8435,4,yes,1024
4,60.000000,60.000000,1781.400000,1781.400000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,40937,42739,40937,Dual Task,3,8907,5,yes,1024
5,66.666667,66.666667,1598.333333,1598.333333,black,xpyriment,legacy,legacy,0,uniform,...,NaN,42987,45280,42987,Dual Task,4,9590,6,yes,1024
6,57.142857,57.142857,1441.571429,1441.571429,black,xpyriment,legacy,legacy,0,uniform,...,NaN,45522,47509,45522,Dual Task,4,10091,7,yes,1024
7,62.500000,62.500000,1336.500000,1336.500000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,47769,49926,47768,Dual Task,5,10692,8,yes,1024
8,66.666667,66.666667,1220.777778,1220.777778,black,xpyriment,legacy,legacy,0,uniform,...,NaN,47769,49926,47768,Dual Task,6,10987,9,yes,1024
9,60.000000,60.000000,1176.500000,1176.500000,black,xpyriment,legacy,legacy,0,uniform,...,NaN,47769,49926,47768,Dual Task,6,11765,10,yes,1024


In [6]:
# allocate data of the dataframe of a specifc column

#total wrong asnwer

total_resp_df_exo = df_exo.filter(regex='total_responses')
total_resp_correct_df_exo = df_exo.filter(regex='total_correct')

dim_df_exo = total_resp_df_exo.size

total_err_exo = total_resp_df_exo.iloc[dim_df_exo-1,0:1].values - total_resp_correct_df_exo.iloc[dim_df_exo-1,0:1].values

print(total_resp_df_exo.iloc[dim_df_exo-1,0:1].values)
print(total_resp_correct_df_exo.iloc[dim_df_exo-1,0:1].values)
print(total_err_exo)

total_resp_exo = df_exo.filter(regex='total_responses')

#total time: sum of times from question to answer

total_time_df_exo = df_exo.filter(regex='total_response_time')
total_time_exo = total_time_df_exo.iloc[dim_df_exo-1,0:1].values/1000 # [s]

print(total_time_exo)

#execution time: total time ascending/descending
exe_time_ad_exo = input("insert the total time for ascending/descendig with exo [s]: ")
float(exe_time_ad_exo)

print(exe_time_ad_exo)

[128]
[112]
[16]
[68.747]
insert the total time for ascending/descendig with exo [s]: 633.83
633.83


In [7]:
#data differences

total_err_resp_diff = total_err_exo - total_err_no_exo
tot_time_diff = total_time_exo - total_time_no_exo
tot_ad_time_diff = float(exe_time_ad_exo) - float(exe_time_ad_no_exo)

print(total_err_resp_diff)
print(tot_time_diff)
print(tot_ad_time_diff)

[0]
[0.]
93.71000000000004


In [9]:
# save to .csv
f_metrics_HR_df = pd.DataFrame({'total_errors (no exo, exo, difference)': [total_err_no_exo, total_err_exo, total_err_resp_diff],
                                'total_time_resp (no exo, exo, difference)': [total_time_no_exo, total_time_exo, tot_time_diff],
                                'total_time_ad (no exo, exo, difference)': [tot_ad_time_diff, exe_time_ad_no_exo, exe_time_ad_exo]})

save_name = 'HR_metrics_STEPbySTEP_' + save_name_exo + '.csv'
print(save_name)

f_metrics_HR_df.to_csv(save_name,index=False,header=True)

HR_metrics_STEPbySTEP_XSPINE.csv
